In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_json(r"C:\Users\39331\Desktop\integrato_DEFINITIVO.json", lines = True)

In [3]:
def aggregate_genres(row):
    for i in range(0, len(row)):
        if row[i]['name'] in ['Action', 'Action-Adventure', 'Adventure']:
            row[i]['name'] = 'Action-Adventure'
        if row[i]['name'] in ['Baseball', 'Basketball', 'Bowling', 'Boxing', 
                         'Fishing', 'Football', 'Golf', 'Hockey', 'Pinball',
                        'Skateboarding', 'Soccer', 'Sports', 'Tennis', 'Wrestling',
                        'Driving/Racing', 'Fitness']:
            row[i]['name'] = 'Sports'
        if row[i]['name'] in ['Dual-Joystick Shooter', 'First-Person Shooter', "Shoot 'Em Up",
                        "Shooter"]:
            row[i]['name'] = "Shooter"
        if row[i]['name'] in ['Brawler', 'Fighting', 'Vehicular Combat']:
            row[i]['name'] = "Fighting"
        if row[i]['name'] in ['Flight Simulator', 'Simulation']:
            row[i]['name'] = "Simulation"
        if row[i]['name'] in ['MMORPG', 'Role-Playing']:
            row[i]['name'] = 'Role-Playing'
        if row[i]['name'] in ['Puzzle', 'Trivia/Board Game']:
            row[i]['name'] = "Puzzle"
        if row[i]['name'] in ['Real-Time Strategy', 'Strategy']:
            row[i]['name'] = "Strategy"
    return(row)

In [4]:
df['genere_grouped'] = df['genres'].apply(aggregate_genres)

In [5]:
df['age_rating_one'] = df['age_ratings'].apply(lambda x: x[0])

In [6]:
df = df.drop(columns = ['age_ratings', 'developers', 'genres', 'giantbomb_id', 'name', 
                        'norm_name', 'normalized_name', 'release_dates', 'similar_games'])

In [7]:
df = df.rename(columns = {'name_g':'name', 'genere_grouped':'genres', 'age_rating_one':'age_rating',
                         'similar_games_g':'similar_games'})

In [8]:
def aggregate_age_rating(row):
    if row in ['Three', 'Seven', 'E', 'E10']:
        return("giochi per tutti")
    if row in ['T', 'Twelve', 'Sixteen']:
        return("giochi per adolescenti")
    if row in ['M', 'AO', 'Eighteen']:
        return("giochi per adulti")
    if row in ['RP']:
        return("RP")

In [9]:
df['age_rating'] = df['age_rating'].apply(aggregate_age_rating)

In [10]:
def group_num_char(row):
    if row == 0:
        return("0")
    if row >=1 and row <= 5:
        return("da 1 a 5")
    if row >= 5 and row <= 10:
        return("da 5 a 10")
    if row >= 10 and row <= 20:
        return("da 10 a 20")
    if row > 20:
        return("più di 20")

In [11]:
df['characters'] = df['characters'].apply(group_num_char)

In [12]:
def get_list_game_modes(row):
    try:
        lista = []
        for i in row:
            lista.append(i['name'])
        return(lista)
    except:
        return(row)

In [13]:
df['game_modes'] = df['game_modes'].apply(get_list_game_modes)

In [14]:
def conv_name(row):
    lista = []
    for i in row:
        if i in ['Battle Royale', 'Co-operative', 
                'Massively Multiplayer Online (MMO)', 'Split screen']:
            lista.append("Multiplayer")
        else:
            lista.append(i)
    return(lista)

In [15]:
df['game_modes'] = df['game_modes'].apply(conv_name)

In [16]:
def group_game_modes(row):
    if len(row) == 1:
        if row[0] == "Single player":
            return("Gioco solo Single Player")
        else:
            return("Gioco solo Multiplayer")
    if len(row) > 1:
        if "Single player" in row:
            return("Gioco sia Single Player che Multiplayer")
        else:
            return("Gioco solo Multiplayer")

In [17]:
df['game_modes'] = df['game_modes'].apply(group_game_modes)

In [18]:
df['anno_pubblicazione'] = df['original_release_date'].apply(lambda x: pd.to_datetime(x).year)

In [19]:
def group_year(row):
    if row <= 1999:
        return("gioco uscito prima del 2000")
    if row >= 2000 and row <=2009:
        return("2000-2009")
    if row >= 2010 and row <= 2015:
        return("2010-2015")
    if row >= 2016 and row <= 2019:
        return("2016-2019")
    if row >= 2020:
        return("giochi usciti quest anno")

In [20]:
df['original_release_date'] = df['anno_pubblicazione'].apply(group_year)

In [21]:
smartphone = []
for i in open(r"C:\Users\39331\Desktop\Smartphone.txt"):
    smartphone.append(i.strip("\n"))

In [22]:
PC = []
for i in open(r"C:\Users\39331\Desktop\PC.txt"):
    PC.append(i.strip("\n"))

In [23]:
Console_port = []
for i in open(r"C:\Users\39331\Desktop\Console Portatili.txt"):
    Console_port.append(i.strip("\n"))

In [24]:
def group_platform(row):
    lista = []
    for i in row:
        if i['name'] in smartphone:
            lista.append("Smartphone")
        if i['name'] in PC:
            lista.append("PC")
        if i['name'] in Console_port:
            lista.append("Console Portatili")
        if i['name'] not in smartphone and i['name'] not in PC and i['name'] not in Console_port:
            lista.append("Console")
    return(lista)

In [25]:
df['platform_grouped'] = df['platforms'].apply(group_platform)

In [26]:
def remove_duplicates(row):
    new_row = list(dict.fromkeys(row))
    return(new_row)

In [27]:
df['platform_grouped_unique'] = df['platform_grouped'].apply(remove_duplicates)

In [28]:
def funz_descr (df):
   if df['platform_grouped_unique'] == ["PC"] :
      return 'giochi solo per PC'
   if df['platform_grouped_unique'] == ["Console"] :
      return 'giochi solo per Console'
   if df['platform_grouped_unique'] == ["Console Portatili"] :
      return 'giochi solo per Console Portatili'
   if df['platform_grouped_unique'] == ["Smartphone"] :
      return 'giochi solo per Smartphone'
    
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console"]) :
      return 'giochi per PC e Console'
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Smartphone"]) :
      return 'giochi per PC e Smartphone'
   if sorted(df['platform_grouped_unique']) == sorted(["Smartphone", "Console"]) :
      return 'giochi per Console e Smartphone'
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console Portatili"]) :
      return 'giochi per PC e Console Portatili'
   if sorted(df['platform_grouped_unique']) == sorted(["Console", "Console Portatili"]) :
      return 'giochi per Console e Console Portatili'
   if sorted(df['platform_grouped_unique']) == sorted(["Smartphone", "Console Portatili"]) :
      return 'giochi per Smartphone e Console Portatili'    
    
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console", "Smartphone"]) :
      return 'giochi per PC, Console e Smartphone'
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console", "Console Portatili"]) :
      return 'giochi per PC, Console e Console Portatili'
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console Portatili", "Smartphone"]) :
      return 'giochi per PC, Console Portatili e Smartphone'
   if sorted(df['platform_grouped_unique']) == sorted(["Console Portatili", "Console", "Smartphone"]) :
      return 'giochi per Console, Console Portatili e Smartphone'
    
   if sorted(df['platform_grouped_unique']) == sorted(["PC", "Console Portatili", "Console", "Smartphone"]) :
      return 'giochi per tutte le piattaforme: PC, Console, Console Portatili e Smartphone' 
    
   return 'Other'

In [29]:
df_col = pd.read_json(r"C:\Users\39331\Desktop\sample.json", lines = True)

In [30]:
df_col['platform_description'] = df.apply (lambda madda: funz_descr(madda), axis=1)

In [31]:
df['platforms'] = df_col['platform_description']

In [32]:
df = df.drop(columns = ['time_to_beat', 'platform_grouped',
                       'platform_grouped_unique'])

In [33]:
df.head()

,characters,dlcs,game_modes,name,original_release_date,platforms,similar_games,themes,views,genres,age_rating,anno_pubblicazione
0,più di 20,0,Gioco sia Single Player che Multiplayer,Fortnite,2016-2019,"giochi per PC, Console e Smartphone","[{'name': 'CastleMiner'}, {'name': 'Minecraft'...","[{'name': 'Horror'}, {'name': 'Fantasy'}, {'na...","[{'timestamp': '2020-07-02 08:30:53', 'visuals...",[{'name': 'Shooter'}],giochi per adolescenti,2017
1,più di 20,0,Gioco sia Single Player che Multiplayer,League of Legends,2000-2009,giochi solo per PC,"[{'name': 'Heroes of Newerth'}, {'name': 'Infi...",[{'name': 'Fantasy'}],"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",[{'name': 'MOBA'}],giochi per adolescenti,2009
2,da 5 a 10,0,Gioco solo Multiplayer,Valorant,giochi usciti quest anno,giochi solo per PC,"[{'name': 'Team Fortress 2'}, {'name': 'Overwa...",[{'name': 'Sci-Fi'}],"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",[{'name': 'Shooter'}],giochi per adolescenti,2020
3,da 1 a 5,0,Gioco sia Single Player che Multiplayer,Call of Duty: Modern Warfare,2016-2019,giochi per PC e Console,None,[{'name': 'Modern Military'}],"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",[{'name': 'Shooter'}],giochi per adulti,2019
4,0,1,Gioco solo Multiplayer,Counter-Strike: Global Offensive,2010-2015,giochi per PC e Console,"[{'name': 'Call of Duty: Modern Warfare 3'}, {...",[{'name': 'Modern Military'}],"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",[{'name': 'Shooter'}],giochi per adulti,2012


In [34]:
df.to_json(r"C:\Users\39331\Desktop\Integrato_cleaned.json", lines = True, orient = "records")